# Overview of the asignment
TF I'm trying to achieve here?

# Required libraries

In [1]:
import numpy
import pandas
import plotly.express as px
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix
#Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Creating the Data table

In [2]:
#Three columns are saved as string datatype, however they contain float data that can be useful for the model
#This motivates us to change these columns form string into float
task_data = pandas.read_csv('task_data.csv', delimiter=',')
print(task_data.dtypes, "\n")
for string in ('CTR - Cardiothoracic Ratio','Inscribed circle radius','Heart perimeter'):
  task_data[string] = task_data[string].str.replace(',','.')
  task_data[string] = task_data[string].astype(float)
print(task_data.dtypes, "\n")
task_data

FileNotFoundError: [Errno 2] No such file or directory: 'task_data.csv'

# Preparing data (to make something useful out of this)

In [ ]:
usefulData = task_data[["Heart width", "Lung width", "xx","yy","xy","normalized_diff","Inscribed circle radius", "Polygon Area Ratio", "Heart perimeter", "Heart area ","Lung area" ]]
usefulData

In [ ]:
result = task_data['Cardiomegaly']
result

# Dividing data into train and test

In [ ]:
trainData, testData, trainResult, testResult = train_test_split(usefulData, result, test_size=0.2, random_state=69)

# Weird shit starts here
No one will stop me from optimising everything

# WE CAN GO BIGGER
We'll apply all possible methods, because no one will stop me

# **Universal Optimatisation Functions**

In [ ]:
def pipeOptimiser(classificator, parameters):
  rskf = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=20,
    random_state=None
  )

  pipe = Pipeline(steps=[
      ("scaler", StandardScaler()),
      ("model", classificator())
  ])
  test = classificator()
  grid_search = GridSearchCV(
    estimator=pipe,

    param_grid=parameters,
    scoring="accuracy",
    cv=rskf,
    verbose=1,
    n_jobs=-1
  )
  grid_search.fit(trainData, trainResult)
  return ( grid_search.best_params,grid_search.best_score_)

#K - nearest neighbors



In [ ]:
#Finding best parameters for K - nearest neighbors
param_grid = {
    "model__n_neighbors": [1,2,3,4],
    "model__weights": ["uniform", "distance"],
    "model__metric": ["minkowski", "manhattan", "euclidean", "chebyshev"],
}
best_params_pipe_knn, best_score = pipeOptimiser(KNeighborsClassifier, param_grid)
print(f"Best parameters: {best_params_pipe_knn}")
print(f"Best accuracy (averaged CV): {best_score:.4f}")

In [ ]:
#Testing the train data set with found parameters
pipe_knn = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", KNeighborsClassifier(
        n_neighbors = best_params_pipe_knn['model__n_neighbors'],
        weights     = best_params_pipe_knn['model__weights'],
        metric      = best_params_pipe_knn['model__metric'],
    ))
])
pipe_knn.fit(trainData, trainResult)
cv_score = numpy.round(cross_val_score(pipe_knn, trainData, trainResult), 2)
print("Scores of training data cross-validation (each fold):")
list(map(print, cv_score))
print(f"\nCross-validation mean score: {numpy.mean(cv_score):.3}")
print(f"Standard deviation of CV score: {numpy.std(cv_score):.3f}")

# Support Vector Machine

In [ ]:
#Finding best parameters for Support Vector Machine
param_grid = {
    "model__kernel":["linear", "poly", "rbf", "sigmoid"],
    "model__C":[1,2,3,4],
    "model__gamma":["scale","auto"],
    "model__class_weight":["balanced",None]
}
best_params_pipe_svc, best_score = pipeOptimiser(SVC, param_grid)
print(f"Best parameters: {best_params_pipe_svc}")
print(f"Best accuracy (averaged CV): {best_score:.4f}")

In [ ]:
pipe_svc = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", SVC(                  # Support Vector Classifier
        kernel=best_params_pipe_svc["model__kernel"],               # RBF kernel captures non-linear decision boundaries
        C=best_params_pipe_svc["model__C"],                        # Regularization strength (higher = tighter fit to training data)
        gamma=best_params_pipe_svc["model__gamma"],              # Kernel width; 'scale' adapts to data variance
        class_weight=best_params_pipe_svc["model__class_weight"]           # Treat classes equally (no re-weighting)
    ))
])

pipe_svc.fit(trainData, trainResult)

cv_score = numpy.round(cross_val_score(pipe_svc, trainData, trainResult), 2)

print("Scores of training data cross-validation (each fold):")
list(map(print, cv_score))
print(f"\nCross-validation mean score: {cv_score.mean():.3f}")
print(f"Standard deviation of CV score: {cv_score.std():.3f}")

# Logistic Regression

In [ ]:
#Finding best parameters for Logistic Regression
param_grid = {
  "model__C":[1,2,3],
  "model__penalty":["l1", "l2", "elasticnet", None],
  "model__solver":["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"],
  "model__max_iter":[1000,10000],
  "model__class_weight":["balanced",None],
}
best_params_pipe_log, best_score = pipeOptimiser(LogisticRegression, param_grid)
print(f"Best parameters: {best_params_pipe_log}")
print(f"Best accuracy (averaged CV): {best_score:.4f}")

In [ ]:
pipe_log = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(
        C=best_params_pipe_log["model__C"],
        penalty=best_params_pipe_log["model__penalty"],
        solver=best_params_pipe_log["model__solver"],
        max_iter=best_params_pipe_log["model__max_iter"],
        class_weight=best_params_pipe_log["model__class_weight"]
    ))
])

pipe_log.fit(trainData, trainResult)

cv_score = numpy.round(cross_val_score(pipe_log, trainData, trainResult), 2)

print("Scores of training data cross-validation (each fold):")
list(map(print, cv_score))
print(f"\nCross-validation mean score: {cv_score.mean():.3f}")
print(f"Standard deviation of CV score: {cv_score.std():.3f}")

# Running the Model on the Test Dataset

In [ ]:
#Predicting for all models
y_pred_knn  = pipe_knn.predict(testData)
y_pred_svc  = pipe_svc.predict(testData)
y_pred_log  = pipe_log.predict(testData)

#Evaluating all models
#Accuracy
acc_knn  = accuracy_score(testResult, y_pred_knn)
acc_svc  = accuracy_score(testResult, y_pred_svc)
acc_log  = accuracy_score(testResult, y_pred_log)
#Precision
prec_knn  = precision_score(testResult, y_pred_knn)
prec_svc  = precision_score(testResult, y_pred_svc)
prec_log  = precision_score(testResult, y_pred_log)
#Recall
recall_knn  = recall_score(testResult, y_pred_knn)
recall_svc  = recall_score(testResult, y_pred_svc)
recall_log  = recall_score(testResult, y_pred_log)
#F1
f1_knn  = f1_score(testResult, y_pred_knn)
f1_svc  = f1_score(testResult, y_pred_svc)
f1_log  = f1_score(testResult, y_pred_log)
print(f"Accuracy on test set:")
print(f"- Accuracy of KNN Classifier model on test dataset:                     {acc_knn:.4f}")
print(f"- Accuracy of SVC model on test dataset:                                {acc_svc:.4f}")
print(f"- Accuracy of Logistic Regression model on test dataset:                {acc_log:.4f}")
print(f"\n Precision on test set:")
print(f"- Precision of KNN Classifier model on test dataset:                    {prec_knn:.4f}")
print(f"- Precision of SVC model on test dataset:                               {prec_svc:.4f}")
print(f"- Precision of Logistic Regression model on test dataset:               {prec_log:.4f}")
print(f"\n Recall on test set:")
print(f"- Recall of KNN Classifier model on test dataset:                       {recall_knn:.4f}")
print(f"- Recall of SVC model on test dataset:                                  {recall_svc:.4f}")
print(f"- Recall of Logistic Regression model on test dataset:                  {recall_log:.4f}")
print(f"\n F1 on test set:")
print(f"- F1 of KNN Classifier model on test dataset:                           {f1_knn:.4f}")
print(f"- F1 of SVC model on test dataset:                                      {f1_svc:.4f}")
print(f"- F1 of Logistic Regression model on test dataset:                      {f1_log:.4f}")
print(f"Confusion matrix on test data")
print(f"- Confusion matrix of KNN Classifier model on test dataset:             {confusion_matrix(testResult,y_pred_knn)}")
print(f"- Confusion matrix of SVC model on test dataset:                        {confusion_matrix(testResult,y_pred_svc)}")
print(f"- Confusion matrix of Logistic Regression model on test dataset:        {confusion_matrix(testResult,y_pred_log)}")